### Steps for ROI Extraction
We will start with adjusting the color space
From some EDA, I have found LAB, LUV and YUV colors spaces to be useful for ROI extraction task

Here's the current pipeline
- Convert RGB image into another color space (possibly LAB)
- Extract L channel and apply enhancement using `convertScaleAbs` function from cv2 [alpha = 2.0, beta=-50]
- Dilate image to increase ROI area
- Apply binary threshold on the image with threshold value of 200
- Applying `findContours` on the thresholded image
- Fetch each contours area and sort in descending order.
- Pick highest contour area objects and create bounding boxes using `boundingRect`
- These will be used as ROIs for extraction of hot liquid

- Need to apply some kind of technique to identify how many objects are there in the frame. 
    - Also, how do we know the ROI is actually a coffee mug or glass or something else.
        Template Matching, or maybe some other technique. 
            Some kind of edge detection


In [11]:
import cv2
from PIL import Image
import os
import glob
import matplotlib.pyplot as plt
import numpy as np
import imageio as io
import pandas as pd
import yaml
import sys
from matplotlib import patches
from pycocotools import mask as M
from pprint import pprint
sys.path.append('../scripts')
import utils

In [9]:
with open('../config.yaml','r') as f:
    config = yaml.safe_load(f)
    
img_paths = sorted(glob.glob(config['dataset']['images']+'/img_thermal_*'))

In [13]:
pprint(img_paths[0])

('D:/semester 1 study/Goettingen study material/Practical Course Data '
 'Fusion/dataset/images\\img_thermal_1709304217421.jpg')


In [14]:
color_convs = {
    'HLS':cv2.COLOR_RGB2HLS,
    'HSV': cv2.COLOR_RGB2HSV,
    'LAB':cv2.COLOR_RGB2LAB,
    'LUV': cv2.COLOR_RGB2LUV,
    'YUV': cv2.COLOR_RGB2YUV
}

In [22]:
def get_enhanced_image(image_path, apply_ce=False, apply_blur=False, apply_clahe=False, apply_dilation=False):
    _, c_img = utils.extract_image_and_spectrum(utils.rotate_to_vertical(image_path))
    og_img = c_img
    c_img = cv2.cvtColor(c_img, cv2.COLOR_RGB2GRAY)
    
    if apply_clahe:
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(32,32))
        c_img = clahe.apply(c_img)
    
    if apply_ce:
        c_img = cv2.equalizeHist(c_img)
    
    if apply_blur:
        c_img = cv2.GaussianBlur(c_img, (51, 51), 15)

    cvt_img = cv2.cvtColor(og_img, cv2.COLOR_RGB2HSV)
    
    img_dilate = None
    
    enhanced_img = cv2.convertScaleAbs(cvt_img[...,-1], alpha=2,beta=-40)
    if apply_dilation:
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))  # Adjust kernel size as needed
        img_dilate = cv2.dilate(enhanced_img, kernel, iterations=2)
    
    return c_img, og_img, cvt_img, enhanced_img, img_dilate

In [39]:
c_img, og_img, cvt_img, enhanced_img, img_dilate = get_enhanced_image(image_path=img_paths[0], apply_dilation=True)

In [ ]:
def get_threshold_img(img, t_value=200, apply_open=False, apply_erode_n_dilate=False):
    _, img = cv2.threshold(img_dilate, t_value, 255, cv2.THRESH_BINARY)
    
    if apply_open:
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (31, 31))  # Adjust kernel size as needed
        opened = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel)
        img = opened
        
    if apply_erode_n_dilate:
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (6, 6))  # Adjust kernel size as needed
        eroded = cv2.erode(img, kernel, iterations=6)
        dilated = cv2.dilate(eroded, kernel, iterations=4)
        img = dilated

    return img

In [ ]:
def get_contours_n_bbox(img):
    contours, _ = cv2.findContours(img.astype(np.uint8), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    new_contours = [np.squeeze(c, axis=1) for c in contours]

    contour_areas = pd.DataFrame([(int(i), cv2.contourArea(c)) for i, c in enumerate(contours)])
    contour_areas.columns = ['index', 'area']
    contour_areas.sort_values('area', ascending=False, inplace=True)
    contour_areas['index'] = contour_areas['index'].astype(int)
    
    bboxes = [cv2.boundingRect(new_contours[int(row[0])]) for idx, row in enumerate(contour_areas.iloc[:3].values)]
    
    return bboxes

## PATTERN MATCHING MISSING!
important for identifying if the identified ROI is of a glass/mug or something else hot
    - might have to make use of other channels to see if liquid is in the mug and is possibly less than the height of the cup.